In [ ]:
# !/usr/bin/env python
# coding: utf-8

import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from EDA_functions import *
seed = 5000

In [ ]:
path='../outputs/'
wA_CORR = get_data(name_sav='wA_CORR.sav', path=path)

## 2.3 Poder predictivo.

Es posible tener una idea del poder predictivo de las variables numéricas usando diagramas de violín.

In [ ]:
num_names = list(wA_CORR.select_dtypes('float64').columns)
for name in num_names:
    print(f'################################## VARIABLE: {name} ################################################')
    fig = px.violin(wA_CORR, y=name, x='RainTomorrow', box=True)
    fig.show()

Se puede ver que, en efecto, la mayoría de las veces el comportamiento de la distribución de una variable es distinta cuando se grafica con respecto a la variable objetivo `RainTomorrow`. Este comportamiento es más evidente en variables como `Humidity3pm`, `Humidity9am`,  `Rainfall` y `Pressure9am`, lo cual hace suponer que tienen un poder de discriminación a considerar.

# 3. TRATAMIENTO DE VARIABLES CATEGÓRICA.

## 3.1 Tratamiento de valores perdidos.

Veamos qué variables son categóricas.

In [ ]:
cat_names = list(wA_CORR.select_dtypes('object').columns)

In [ ]:
cat_names

Se toma la decisión de incluir una categoría que represente a los valores perdidos en variables categóricas. Esta categoría llevara el nombre de "Unidentified".

In [ ]:
wA_FILLNA_CATEGORIC = fillna_categoric_data(data=wA_CORR,
                                            list_names=cat_names)

A continuación se dará un vistazo a los tamaños de cada categoría en cada variable, así como a su porcentaje de representación.

In [ ]:
for name in cat_names:
    print(f'################# VARIABLE: {name} #########################################' )
    print(wA_FILLNA_CATEGORIC[name].value_counts())
    print(round((wA_FILLNA_CATEGORIC[name].value_counts(normalize=True))*100), 2)

Observamos que ningún nombre de categoría cae por debajo del 1%, por ende, NO se toma ninguna decisión acerca de fusionar categorías de alguna variable con el fin de evitar categorías demasiado pequeñas.

## 3.2 Información sobre la variable objetivo.

In [ ]:
count_objected = wA_FILLNA_CATEGORIC['RainTomorrow'].value_counts().to_frame().reset_index()
count_objected.columns = ['RainTomorrow', 'count']
fig = px.pie(count_objected, values='count', names='RainTomorrow', title='¿Llovió al día siguiente?')
fig.show()

El gráfico de pastel anterior da cuenta que hay un problema de desbalanceo en la variable objetivo `RainTomorrow`. Es decir, la clase mayoritaría (No llovió) es considerablemente más grande que la clase minoritaria (Sí llovió). 

Se lidiará con este evento más tarde, en el entrenamiento y optimización.

# 4. PEQUEÑA INGENIERÍA DE CARACTERÍSTICAS.

Dado quer estamos ante una data con indexación en el tiempo, podemos rescatar información tal como:
* el mes del año
* la semana del año en la que se ubica el día (del total de 52).

Tales variables pueden ayudar a capturar patrones como el ciclo de lluvia inherentes a través del año en los lugares involucrados.

In [ ]:
wA_FILLNA_CATEGORIC['month'] = wA_FILLNA_CATEGORIC['Date'].dt.month
wA_FILLNA_CATEGORIC['week_of_year'] = wA_FILLNA_CATEGORIC['Date'].dt.isocalendar().week

Cambiemos a tipo categórico las variables recien creadas.

In [ ]:
wA_FILLNA_CATEGORIC['month'] = wA_FILLNA_CATEGORIC['month'].astype('object')
wA_FILLNA_CATEGORIC['week_of_year'] = wA_FILLNA_CATEGORIC['week_of_year'].astype('object')

Cambiemos a tipo binario la naturaleza de la variable objetivo `RainTomorrow`.

In [ ]:
wA_FILLNA_CATEGORIC['RainTomorrow'] = wA_FILLNA_CATEGORIC['RainTomorrow'].map({'Yes': 1, 'No': 0})

# 5. PEQUEÑA SELECCIÓN DE VARIABLES.

## 5.1 Selección de variables categóricas.

Se hará una pequeña selección de varibles numéricas, ANTES DE ENTRENAMIENTO, usando la __información mutua__. Este es un valor no negativo que mide la dependencia entre variables aleatorias. Valores más altos significan una dependencia más alta. 

Entonces, se calculará este valor entre las variables numéricas y la variable objetivo binaria `RainTomorrow`.

In [ ]:
num_names = list(wA_FILLNA_CATEGORIC.select_dtypes('float64').columns)

Se necesita que no haya valores perdidos en variables numéricas. Por ende, se trabajará con una versión de la data sin presencia de valores perdidos en las variables numéricas.

In [ ]:
wA_FILLNA_CATEGORIC_TEMPORARY = wA_FILLNA_CATEGORIC[num_names + ['RainTomorrow']].dropna()
wA_FILLNA_CATEGORIC_TEMPORARY.reset_index(drop=True, inplace=True)

A continuación se muestra el porcentaje de la data que ha quedado.

In [ ]:
print('Porcentaje de data después de eliminación de filas con misssing: ', round((wA_FILLNA_CATEGORIC_TEMPORARY.shape[0] / wA_FILLNA_CATEGORIC.shape[0]) * 100, 2), '%')

A continuación se exhibe la configuación de la variable objetivo `RainTomorrow` después de missing.

In [ ]:
wA_FILLNA_CATEGORIC_TEMPORARY[['RainTomorrow']].value_counts(normalize=True)

De los dos resultados anteriores, observamos que aún conservamos un porcentaje importante de la data. Además, se observa que la configuración de la variable objetivo, antes y después de missing, es muy similar. Siendo así, vamos a asumir que conservamos representatividad suficiente para tener acceso a las conclusiones en cuanto a la información mutua, usando la data sin missing.

__NOTA:__ El tratamiento definitivo de missing en las variables numéricas, se tendrá en cuenta en el entrenamiento de modelo. Este se puede hacer añadiendo alguna técnica de imputación, tal vez no paramétrica como `KNNImputer`, en el pipeline  de entrenamiento. O bien, puede ser que el modelo ya contemple la presencia de missing en el entrenamiento (por ejemplo `xboost` o `HistGradientBoostingClassifier`.

Calculemos entonces la información mutua.

In [ ]:
mutual_information_score(data=wA_FILLNA_CATEGORIC_TEMPORARY,
                         feature_names=num_names,
                         y_label_name='RainTomorrow',
                         list_bool_True=None,
                         seed=seed)

Observamos que las variables que ofrecen una mayor influencia en el objetivo `RainTomorrow` son `Humidity3pm`, `Rainfall` y `Humidity9am`, las cuales tiene que ver con la precipitación y la humedad. Lo anterior, va muy en sintonía con la conclusión de poder de discriminación observado en los diagramas de violín.

Las que ejercen menor influencia, es la que tiene que ver con la temperatura,`MinTemp`, o bien con la rapidez del viento a las 9 am y a las 3pm, `WindSpeed9am` y `WindSpeed3pm`.

Decidimos eliminar la variable de menor influencia, es decir, `WindSpeed9am`.

In [ ]:
mut_info_names = ['WindSpeed9am']
wA_MUTUAL_INFO = wA_FILLNA_CATEGORIC.drop(mut_info_names, axis=1)

## 5.2 Sobre las variables categóricas.

Para este ejercicio, se usarán todas las variables categóricas. 

La importancia de ellas, y también de las numéricas, pueden estudiarse  DESPUÉS DE ENTRENAMIENTO, usando las __feature_importances__ ofrecidas por los estimadores de la librería `scikit-learn`. O  bien, si éstas no se dan, las importancias pueden calcularse usando las __shape_feature_importances__ ofrecicidas por librerías como `shap`, y que se construyen considerando las __shap-values__. Una vez conocidas estas importancias, se puede hacer una selección de variables y reiniciar el entrenamiento.

Por supuesto, la selección de variables, tanto de variables categóricas como numéricas, puede llegar a ser un proceso dinámico y que se puede hacer de manera conjunta en una combinación de técnicas, esto antes y después de entrenamiento.

En cuanto a esta prueba. Sólo se hará selección de variables numéricas antes de entrenamiento.

# 6. MANEJO DE LA CRONOLOGÍA y DIVISIÓN DE LA DATA.

Consigamos la información que se obtiene en cada año y hagamos el conteo correspondiente.

In [ ]:
wA_MUTUAL_INFO['year'] = wA_MUTUAL_INFO['Date'].dt.year

In [ ]:
count_years = wA_MUTUAL_INFO['year'].value_counts().to_frame().reset_index()
count_years.columns = ['year', 'count']

In [ ]:
fig = px.bar(count_years, x='year', y='count')
fig.show()

Para poder contemplar de alguna manera la naturaleza temporal de la data, la división entre los conjuntos de __Entrenamiento__, __Validación__ y __Prueba__ no podrá ser aleatoria, sino que tendrá que contemplar una estructura de *pasado* y *futuro*.

Por ende, se establece la división de la data teniendo en cuenta al año 2015 como umbral de separación.

In [ ]:
wA_train = wA_MUTUAL_INFO[wA_MUTUAL_INFO['year'] <= 2015]
wA_test = wA_MUTUAL_INFO[wA_MUTUAL_INFO['year'] > 2015]

Por ende, se obtienen los siguientes tamaños.

In [ ]:
wA_train

In [ ]:
wA_test

Así quedan los porcentajes de representación con respecto a la data total.

In [ ]:
print('Representación del CONJUNTO DE ENTRENAMIENTO con respecto al total:', round((wA_train.shape[0]/wA_FILLNA_CATEGORIC.shape[0])*100, 2), '%')

In [ ]:
print('Representación del CONJUNTO DE PRUEBA con respecto al total:', round((wA_test.shape[0]/wA_FILLNA_CATEGORIC.shape[0])*100, 2), '%')

Los siguientes diagramas de pastel dan cuenta de que la configuración binaria original de la variable objetivo `RainTomorrow`, que se tiene en la data total, ha logrado heredarse a las contrapartes de __Entrenamiento__ y __Prueba__ (es decir, son muy similares entre ellas). El primer pastel se refiera a la data total, el segundo al __Entrenamiento__ y el tercero al de __Prueba__.

In [ ]:
for data in [wA_MUTUAL_INFO, wA_train, wA_test]:
    count_objected = data['RainTomorrow'].value_counts().to_frame().reset_index()
    count_objected.columns = ['RainTomorrow', 'count']
    fig = px.pie(count_objected, values='count', names='RainTomorrow', title='¿Llovió al día siguiente?')
    fig.show()

No se contempla un tercer conjunto de __Validación__, porque se considera que la cantidad de data es de tal tamaño que bastará con monitoriar las medidas de cross-validación en el conjunto de __Entrenamiento__.

# 7. LISTA FINAL DE FEATURES.

Se exihiben qué variables conformarán el conjunto definitivo de features. Están serán puestas en un archivo de confifguración *config.yaml* para un manejo más flexible en los scripts de entrenamiento y optimización.

In [ ]:
wA_MUTUAL_INFO.columns

In [ ]:
objective_name = 'RainTomorrow'

In [ ]:
feratures_names = ['Location', 'MinTemp', 'Rainfall', 'WindGustDir',
                   'WindGustSpeed', 'WindDir9am', 'WindDir3pm', 'WindSpeed3pm',
                   'Humidity9am', 'Humidity3pm', 'Pressure9am',
                   'RainToday',  'month', 'week_of_year'],

# 8. GUARDADO DE LA DATA.

Los conjuntos de datos de __Entrenamiento__ y __Prueba__ (más bien los dataframes) se guardarán en archivos binarios.

In [ ]:
pickle.dump(wA_train, open('../outputs/wA_train.sav', 'wb'))
pickle.dump(wA_test, open('../outputs/wA_test.sav', 'wb'))